In [101]:
import wrds
import pandas as pd
import numpy as np
import os

print('Connexion à WRDS...')
db = wrds.Connection(wrds_username='gregcouts1')

# Paramètres
start_date = '2003-01-01'
end_date = '2023-12-31'
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 1. Extraire tous les tickers S&P 500 avec market-cap et secteur, présents sur toute la période
sp500_query = f"""
    SELECT DISTINCT a.permno, b.ticker, b.shrcd, b.exchcd, b.namedt, b.nameenddt, b.siccd, b.ncusip
    FROM crsp.msp500list a
    JOIN crsp.stocknames b ON a.permno = b.permno
    WHERE a.start <= '{start_date}'
    AND a.ending >= '{end_date}'
"""
sp500_df = db.raw_sql(sp500_query)

# Récupérer market-cap mensuel pour chaque permno
mcap_query = f"""
    SELECT permno, date, prc, shrout, abs(prc)*shrout*0.001 as mcap_musd
    FROM crsp.msf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""
mcap_df = db.raw_sql(mcap_query, date_cols=['date'])
mcap_df = mcap_df.sort_values(['permno','date'])
mcap_last = mcap_df.groupby('permno').last().reset_index()

# Merge market-cap et tickers
sp500_df = pd.merge(sp500_df, mcap_last[['permno','mcap_musd']], on='permno', how='left')

# Exclure les firmes sous le 30e percentile de market-cap NYSE
nyse_permnos = sp500_df[sp500_df['exchcd']==1]['permno'].unique()
nyse_mcaps = sp500_df[sp500_df['permno'].isin(nyse_permnos)]['mcap_musd'].dropna()
mcap_cutoff = np.percentile(nyse_mcaps, 30)
sp500_df = sp500_df[sp500_df['mcap_musd'] >= mcap_cutoff]

# Exclure les industries Fama-French (real estate, coal, steel, mines, oil, gold)
excluded_sic = [65, 12, 33, 10, 13, 10] # Real estate, coal, steel, mines, oil, gold (approximate SIC codes)
sp500_df = sp500_df[~sp500_df['siccd'].astype(str).str[:2].isin([str(s) for s in excluded_sic])]

# 2. Extraction Compustat (EPS trimestriel) pour tous les tickers restants
compustat_data = []
for ticker in sp500_df['ticker'].unique():
    eps_query = f"""
        SELECT datadate, epspxq, tic
        FROM comp.fundq
        WHERE tic = '{ticker}'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'
        AND popsrc = 'D'
        AND consol = 'C'
        AND datadate >= '{start_date}'
        AND datadate <= '{end_date}'
        ORDER BY datadate
    """
    eps_df = db.raw_sql(eps_query, date_cols=['datadate'])
    if eps_df.empty:
        continue
    eps_df['ticker'] = ticker
    compustat_data.append(eps_df)
if compustat_data:
    df_comp = pd.concat(compustat_data, ignore_index=True)
else:
    df_comp = pd.DataFrame(columns=['datadate','epspxq','ticker'])

# 3. Extraction CRSP (prix quotidiens) pour ces tickers
crsp_data = []
for ticker in sp500_df['ticker'].unique():
    permnos = sp500_df[sp500_df['ticker']==ticker]['permno']
    if permnos.empty:
        continue
    permno = permnos.iloc[0]
    price_query = f"""
        SELECT date, prc
        FROM crsp.dsf
        WHERE permno = {permno}
        AND date BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY date
    """
    price_df = db.raw_sql(price_query, date_cols=['date'])
    if price_df.empty:
        continue
    price_df['price'] = price_df['prc'].abs()
    price_df['ticker'] = ticker
    crsp_data.append(price_df)
if crsp_data:
    df_crsp = pd.concat(crsp_data, ignore_index=True)
else:
    df_crsp = pd.DataFrame(columns=['date','prc','price','ticker'])

# Merge direct des datasets sur ticker et date (sans backward fill EPS)
df_crsp['date'] = pd.to_datetime(df_crsp['date'], errors='coerce')
df_comp['datadate'] = pd.to_datetime(df_comp['datadate'], errors='coerce')
df_crsp['ticker'] = df_crsp['ticker'].astype(str)
df_comp['ticker'] = df_comp['ticker'].astype(str)

merged = pd.merge(df_crsp, df_comp, left_on=['ticker','date'], right_on=['ticker','datadate'], how='left')

final = merged[['date','ticker','price','epspxq']].copy()
output_path = os.path.join(os.getcwd(), 'WRDS_extract.csv')
final.to_csv(output_path, index=False)
print("\n✓ Données extraites et formatées : prix quotidiens et EPS trimestriel (merge direct, sans backward fill).")
print(final.head())
print(f"\n✓ Fichier CSV sauvegardé : {output_path}")

Connexion à WRDS...
Loading library list...
Loading library list...
Done
Done


C:\Users\grego\AppData\Local\Temp\ipykernel_21040\2067690016.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comp = pd.concat(compustat_data, ignore_index=True)



✓ Données extraites et formatées : prix quotidiens et EPS trimestriel (merge direct, sans backward fill).
        date ticker  price  epspxq
0 2003-01-02    MMC  47.59    <NA>
1 2003-01-03    MMC  47.45    <NA>
2 2003-01-06    MMC  48.82    <NA>
3 2003-01-07    MMC  48.73    <NA>
4 2003-01-08    MMC  47.84    <NA>

✓ Fichier CSV sauvegardé : c:\Users\grego\OneDrive\Documents\GitHub\Devoir-2\WRDS_extract.csv


In [118]:
# Fermer explicitement la connexion WRDS si elle existe
try:
    db.close()
    print('Connexion WRDS fermée avec succès.')
except Exception as e:
    print(f'Erreur lors de la fermeture de la connexion WRDS : {e}')

Connexion WRDS fermée avec succès.


In [110]:
# Forward fill EPS from WRDS_extract.csv, nettoyage, suppression des lignes sans EPS filled, exclusion EPS=0, renommage colonnes
import pandas as pd
import numpy as np
csv_path = 'WRDS_extract.csv'
df = pd.read_csv(csv_path, parse_dates=['date'])
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
# Forward fill EPS pour chaque ticker
df['BPA'] = df.groupby('ticker')['epspxq'].fillna(method='ffill')
# Supprimer les stocks qui n'ont jamais de BPA (même après ffill)
tickers_with_bpa = df.groupby('ticker')['BPA'].apply(lambda x: x.notna().any())
df = df[df['ticker'].isin(tickers_with_bpa[tickers_with_bpa].index)]
# Supprimer les lignes sans BPA
df = df[df['BPA'].notna()]
# Exclure les stocks à BPA = 0
df = df[df['BPA'] != 0]
# Calculer P/B glissant
df['P/B glissant'] = np.where(df['BPA'].notna(), df['price']/df['BPA'], np.nan)
# Sauver le résultat (stock_random.csv) sans epspxq
output_path = 'stock_random.csv'
df[['date','ticker','price','BPA','P/B glissant']].to_csv(output_path, index=False)
print(f"✓ Nettoyage, forward fill, exclusion BPA=0, renommage et export terminés. Fichier sauvegardé : {output_path}")
print(df[['date','ticker','price','BPA','P/B glissant']].head())

C:\Users\grego\AppData\Local\Temp\ipykernel_21040\2705461045.py:8: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['BPA'] = df.groupby('ticker')['epspxq'].fillna(method='ffill')
C:\Users\grego\AppData\Local\Temp\ipykernel_21040\2705461045.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['BPA'] = df.groupby('ticker')['epspxq'].fillna(method='ffill')


✓ Nettoyage, forward fill, exclusion BPA=0, renommage et export terminés. Fichier sauvegardé : stock_random.csv
         date ticker  price   BPA  P/B glissant
20 2003-01-31      A  16.48 -0.24    -68.666667
21 2003-02-03      A  16.49 -0.24    -68.708333
22 2003-02-04      A  16.30 -0.24    -67.916667
23 2003-02-05      A  16.32 -0.24    -68.000000
24 2003-02-06      A  12.26 -0.24    -51.083333


In [111]:
# Tirage aléatoire réplicable de 50 stocks, création d'un 3e DataFrame et export CSV
import pandas as pd
import numpy as np
df = pd.read_csv('stock_random.csv', parse_dates=['date'])
np.random.seed(42)
tickers = df['ticker'].drop_duplicates().values
n_draw = min(50, len(tickers))
tickers_draw = np.random.choice(tickers, size=n_draw, replace=False)
df_draw = df[df['ticker'].isin(tickers_draw)].copy()
output_draw_path = 'stock_draw_50.csv'
df_draw.to_csv(output_draw_path, index=False)
print(f"✓ Tirage réplicable de {n_draw} stocks terminé. Fichier sauvegardé : {output_draw_path}")
print(f"Tickers tirés au sort ({n_draw}): {tickers_draw}")
print(f"DataFrame filtré shape: {df_draw.shape}")

✓ Tirage réplicable de 50 stocks terminé. Fichier sauvegardé : stock_draw_50.csv
Tickers tirés au sort (50): ['SHW' 'CAT' 'PA' 'C' 'SRE' 'WMB' 'XEL' 'DHR' 'MS' 'ELV' 'AON' 'USB' 'RTX'
 'CTAS' 'GD' 'JNJ' 'BHI' 'NB' 'HES' 'MU' 'AZO' 'APD' 'CCC' 'AXP' 'ALL'
 'AEP' 'INTU' 'MCRN' 'HON' 'EA' 'COST' 'ECL' 'TJX' 'NKE' 'GE' 'SD' 'ABT'
 'LOW' 'SBUX' 'UPS' 'KR' 'NSP' 'BKR' 'SYY' 'SCHW' 'MCO' 'FCX' 'CL' 'PG'
 'BDX']
DataFrame filtré shape: (245675, 5)


In [120]:
# Extraction et fusion des prévisions d'EPS IBES pour les 50 stocks tirés au sort (moyenne highest/lowest) + export CSV
import wrds
db = wrds.Connection(wrds_username='gregcouts1')
df_draw = pd.read_csv('stock_draw_50.csv', parse_dates=['date'])
tickers_draw = df_draw['ticker'].drop_duplicates().tolist()
# Récupérer les prévisions d'EPS IBES pour chaque ticker
ibes_data = []
for ticker in tickers_draw:
    ibes_query = f"""
        SELECT ticker, fpedats, fpi, highest, lowest
        FROM ibes.statsum_epsus
        WHERE ticker = '{ticker}'
        AND fpedats >= '2003-01-01'
        AND fpedats <= '2023-12-31'
        ORDER BY fpedats
    """
    ibes_df = db.raw_sql(ibes_query, date_cols=['fpedats'])
    if not ibes_df.empty:
        ibes_data.append(ibes_df)
if ibes_data:
    df_ibes = pd.concat(ibes_data, ignore_index=True)
    df_ibes['ibes_est'] = df_ibes[['highest','lowest']].mean(axis=1)
else:
    df_ibes = pd.DataFrame(columns=['ticker','fpedats','fpi','highest','lowest','ibes_est'])
# Fusionner la prévision IBES avec le DataFrame tiré
df_draw = df_draw.merge(df_ibes[['ticker','fpedats','ibes_est']], left_on=['ticker','date'], right_on=['ticker','fpedats'], how='left')
# Exporter le DataFrame enrichi dans le CSV
output_draw_path = 'stock_draw_50.csv'
df_draw.to_csv(output_draw_path, index=False)
print("✓ Extraction, fusion et export CSV avec la colonne ibes_est terminés.")
print(df_draw.head())

Loading library list...
Done
Done
✓ Extraction, fusion et export CSV avec la colonne ibes_est terminés.
        date ticker  price   BPA  P/B glissant    fpedats  ibes_est
0 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.545
1 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31      0.53
2 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31      0.53
3 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.575
4 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.575
✓ Extraction, fusion et export CSV avec la colonne ibes_est terminés.
        date ticker  price   BPA  P/B glissant    fpedats  ibes_est
0 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.545
1 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31      0.53
2 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31      0.53
3 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.575
4 2003-03-31    ABT  37.61  0.47     80.021277 2003-03-31     0.575
